# Ver 1: Valid Output+Random Parameters

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from typing import Dict, Tuple, List
import helper
import matplotlib.pyplot as plt

In [8]:
import torch
from torch.utils.data import Dataset
import numpy as np

class LinearModelDataset(Dataset):
    def __init__(self, num_samples: int, nr: int, nc: int, dt: float, F: float):
        self.num_samples = num_samples
        self.nr = nr
        self.nc = nc
        self.dt = dt
        self.F = F
        self.samples = self._generate_samples()

    def _generate_gaussian_field(self, n, nrv, ncv):
        """
        Generate a Gaussian field composed of n Gaussian functions.
        """
        mux = np.random.choice(ncv, n)
        muy = np.random.choice(range(2, nrv - 2), n)
        sigmax = np.random.uniform(1, ncv/4, n)
        sigmay = np.random.uniform(1, nrv/4, n)

        v = np.zeros((nrv, ncv))
        for i in range(n):
            for x in range(ncv):
                for y in range(nrv):
                    # Create three copies for pseudo-periodic field
                    gauss = np.exp(-((x-mux[i])**2/(2*sigmax[i]**2) + (y-muy[i])**2/(2*sigmay[i]**2)))
                    gauss += np.exp(-((x-(mux[i]-ncv))**2/(2*sigmax[i]**2) + (y-muy[i])**2/(2*sigmay[i]**2)))
                    gauss += np.exp(-((x-(mux[i]+ncv))**2/(2*sigmax[i]**2) + (y-muy[i])**2/(2*sigmay[i]**2)))
                    v[y,x] += gauss
        return v

    def _generate_circular_field(self, v):
        """
        Generate a circular field from gradient of input field.
        """
        grad_v_y, grad_v_x = np.gradient(v)
        return -grad_v_y, grad_v_x

    def _generate_params(self):
        """
        Generate model parameters similar to generate_world function.
        """
        # Base grid parameters
        DX_C = torch.ones(self.nr, self.nc + 1)
        DY_C = torch.ones(self.nr + 1, self.nc)
        DX_G = torch.ones(self.nr + 1, self.nc)
        DY_G = torch.ones(self.nr, self.nc + 1)
        RAC = torch.ones(self.nr, self.nc)

        # Generate random diffusivities (must be positive)
        KX = torch.abs(torch.rand(self.nr, self.nc + 1))
        KY = torch.abs(torch.rand(self.nr + 1, self.nc))

        # Generate velocities using Gaussian field approach
        num_gauss = 16  # Number of Gaussian functions for velocity field
        gauss = self._generate_gaussian_field(num_gauss, self.nr + 1, self.nc + 1)
        VX_np, VY_np = self._generate_circular_field(gauss)
        
        # Convert velocities to PyTorch and scale
        VX = torch.from_numpy(100 * VX_np[:-1, :]).float()
        VY = torch.from_numpy(100 * VY_np[:, :-1]).float()

        # Create random forcing term
        f = torch.randn(self.nr * self.nc)

        return {
            'KX': KX,
            'KY': KY,
            'DX_C': DX_C,
            'DY_C': DY_C,
            'DX_G': DX_G,
            'DY_G': DY_G,
            'VX': VX,
            'VY': VY,
            'RAC': RAC,
            'f': f,
        }

    def _generate_samples(self) -> list:
        """
        Generate multiple samples with parameters.
        """
        samples = []
        for _ in range(self.num_samples):
            params = self._generate_params()
            
            x_t = torch.randn(self.nr * self.nc)
            lambda_t_plus_1 = torch.randn(self.nr * self.nc)
            dx_dt_lambda = self._compute_dx_dt_lambda(params, x_t, lambda_t_plus_1)
            
            samples.append((params, x_t, lambda_t_plus_1, dx_dt_lambda))
        
        return samples

    def _compute_dx_dt_lambda(self, params, x_t, lambda_t_plus_1):
        """
        Compute dx(t+1)/dx(t) * lambda(t+1) for the linear model.
        """
        # Convert PyTorch tensors to numpy arrays for helper function
        np_params = {
            key: tensor.cpu().detach().numpy() if torch.is_tensor(tensor) else tensor
            for key, tensor in params.items()
        }
        
        # Get model matrix M using helper function
        M = helper.make_M_2d_diffusion_advection_forcing(
            nr=self.nr,
            nc=self.nc,
            dt=self.dt,
            KX=np_params['KX'],
            KY=np_params['KY'],
            DX_C=np_params['DX_C'],
            DY_C=np_params['DY_C'], 
            DX_G=np_params['DX_G'],
            DY_G=np_params['DY_G'],
            VX=np_params['VX'],
            VY=np_params['VY'],
            RAC=np_params['RAC'],
            F=self.F,
            cyclic_east_west=True,
            cyclic_north_south=False,
            M_is_sparse=False
        )
        
        # Convert to numpy arrays and do matrix multiplication
        lambda_np = lambda_t_plus_1.cpu().detach().numpy()
        result_np = M.T @ lambda_np
        
        # Convert back to PyTorch tensor
        return torch.from_numpy(result_np).float()

    def __len__(self) -> int:
        return self.num_samples

    def __getitem__(self, idx: int):
        return self.samples[idx]

class LinearModelNet(torch.nn.Module):
    def __init__(self, nr: int, nc: int):
        super().__init__()
        self.nr = nr
        self.nc = nc
        
        self.state_size = nr * nc
        self.param_size = (nr * (nc + 1) +
                           (nr + 1) * nc +
                           nr * (nc + 1) +
                           (nr + 1) * nc +
                           (nr + 1) * nc +
                           nr * (nc + 1) +
                           nr * (nc + 1) +
                           (nr + 1) * nc +
                           nr * nc +
                           nr * nc)
        
        self.input_size = self.state_size * 2 + self.param_size
        self.output_size = nr * nc
        
        print(f"Input size: {self.input_size}")
        
        # Adjust the architecture to handle the correct input size
        self.fc1 = torch.nn.Linear(self.input_size, 2048)
        self.fc2 = torch.nn.Linear(2048, 1024)
        self.fc3 = torch.nn.Linear(1024, 512)
        self.fc4 = torch.nn.Linear(512, self.output_size)
        
        self.activation = torch.nn.ReLU()
        
    def forward(self, x: torch.Tensor, lambda_t_plus_1: torch.Tensor, params: Dict[str, torch.Tensor]) -> torch.Tensor:
        batch_size = x.size(0) 
        
        # Ensure all tensors have the correct batch size
        x = x.view(batch_size, -1)
        lambda_t_plus_1 = lambda_t_plus_1.view(batch_size, -1)
        
        # Flatten the params dictionary
        params_flat = torch.cat([param.view(batch_size, -1) for param in params.values()], dim=1)
        
        # Concatenate all inputs
        x = torch.cat([x, lambda_t_plus_1, params_flat], dim=1)
        
        # Add some debugging print statements
        #print(f"x shape: {x.shape}")
        #print(f"Input size: {self.input_size}")
        
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        
        return x

def train_model(model: torch.nn.Module, dataloader: DataLoader, num_epochs: int, learning_rate: float):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for params, x_t, lambda_t_plus_1, dx_dt_lambda in dataloader:
            optimizer.zero_grad()
            output = model(x_t, lambda_t_plus_1, params)
            loss = criterion(output, dx_dt_lambda)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")



Creating datasets...
Input size: 1280
Training started...
Epoch 1/10
Training Loss: 214.6914
Test Loss: 208.5321
Epoch 2/10
Training Loss: 214.5220
Test Loss: 208.5352
Epoch 3/10
Training Loss: 214.4871
Test Loss: 208.5159
Epoch 4/10
Training Loss: 214.3631
Test Loss: 208.4732
Epoch 5/10
Training Loss: 214.5614
Test Loss: 208.4675
Epoch 6/10
Training Loss: 214.1176
Test Loss: 208.5173
Epoch 7/10
Training Loss: 213.5616
Test Loss: 205.8817
Epoch 8/10
Training Loss: 209.3307
Test Loss: 203.1971
Epoch 9/10
Training Loss: 203.5546
Test Loss: 195.4682
Epoch 10/10
Training Loss: 199.1494
Test Loss: 192.1955
Training complete!


Comparison of first 10 values (from test set):
Index    True Value    Predicted   Difference
--------------------------------------------
0         -4.248634    -2.360214    -1.888419
1          4.318857     4.571734    -0.252878
2          1.089023    -1.273595     2.362618
3         -8.295985    -1.941436    -6.354549
4         -7.650617     2.364271   -10.014888
5         -0.869654    -2.634295     1.764641
6         18.564604     4.515879    14.048725
7         -4.560369    -1.983807    -2.576562
8         -6.133597     1.916654    -8.050251
9          3.886673    -1.820285     5.706958

Test Set Statistics:
Mean Absolute Error: 10.683954
Root Mean Square Error: 13.867004
True vector norm: 145.383255
Predicted vector norm: 44.371807


In [9]:
if __name__ == "__main__":
    # Model parameters
    nr, nc = 10, 10
    dt = 0.1
    F = 1.0
    
    # Dataset parameters
    train_samples = 18000  # 90% for training
    test_samples = 200    # 10% for testing
    batch_size = 320
    num_epochs = 10
    learning_rate = 0.001

    print("Creating datasets...")
    # Create datasets
    train_dataset = LinearModelDataset(train_samples, nr, nc, dt, F)
    test_dataset = LinearModelDataset(test_samples, nr, nc, dt, F)
    
    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Create and train the model
    model = LinearModelNet(nr, nc)
    
    # Training loop
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    
    # Lists to store losses for plotting
    train_losses = []
    test_losses = []
    
    print("Training started...")
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for params, x_t, lambda_t_plus_1, dx_dt_lambda in train_dataloader:
            optimizer.zero_grad()
            output = model(x_t, lambda_t_plus_1, params)
            loss = criterion(output, dx_dt_lambda)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            
        # Evaluate on test set
        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for params, x_t, lambda_t_plus_1, dx_dt_lambda in test_dataloader:
                output = model(x_t, lambda_t_plus_1, params)
                test_loss = criterion(output, dx_dt_lambda)
                total_test_loss += test_loss.item()
        
        # Store average losses
        avg_train_loss = total_train_loss/len(train_dataloader)
        avg_test_loss = total_test_loss/len(test_dataloader)
        train_losses.append(avg_train_loss)
        test_losses.append(avg_test_loss)
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Training Loss: {avg_train_loss:.4f}")
        print(f"Test Loss: {avg_test_loss:.4f}")
    
    print("Training complete!")

    # Plot training and testing losses
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs + 1), train_losses, 'b-', label='Training Loss')
    plt.plot(range(1, num_epochs + 1), test_losses, 'r-', label='Test Loss')
    plt.title('Training and Testing Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Evaluate on test examples
    model.eval()
    test_params, test_x, test_lambda, test_true = next(iter(test_dataloader))
    
    # Get model prediction for first test example
    with torch.no_grad():
        predicted = model(test_x[0:1], test_lambda[0:1], 
                        {k: v[0:1] for k, v in test_params.items()})

    # Remove batch dimension
    predicted = predicted.squeeze(0)
    test_true = test_true[0]  # Get first example's true values

    # Print comparison
    print(f"\nComparison of first 10 values (from test set):")
    print(f"{'Index':<6} {'True Value':>12} {'Predicted':>12} {'Difference':>12}")
    print("-" * 44)
    for i in range(10):
        print(f"{i:<6} {test_true[i]:>12.6f} {predicted[i]:>12.6f} {(test_true[i] - predicted[i]):>12.6f}")

    # Print overall statistics
    print(f"\nTest Set Statistics:")
    print(f"Mean Absolute Error: {torch.abs(test_true - predicted).mean():.6f}")
    print(f"Root Mean Square Error: {torch.sqrt(torch.mean((test_true - predicted)**2)):.6f}")
    print(f"True vector norm: {torch.norm(test_true):.6f}")
    print(f"Predicted vector norm: {torch.norm(predicted):.6f}")

    # Plot true vs predicted values
    plt.figure(figsize=(12, 6))
    plt.plot(test_true.numpy(), label='True Values', marker='o')
    plt.plot(predicted.numpy(), label='Predicted Values', marker='x')
    plt.title('True vs Predicted Values (Test Set)')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

### To-do:

- Set things up so we can save data
- Train with more data to see if it can learn effectively

- Try covariance (maybe test model on OOD covariance?)
- Try data from a model simulated over time
- See how well that does on backprop